<a href="https://colab.research.google.com/github/ele9996/keyword-spotting/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile

In [3]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--model', type=str, required=True, help='model name')
#parser.add_argument('--mfcc', action='store_true', help='use MFCCs')
#parser.add_argument('--silence', action='store_true', help='add silence')
#args = parser.parse_args()

mfcc=True
silence=False

In [4]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

#Data extraction from file

In [5]:
#if args.silence is True:
if silence is True:
    data_dir = os.path.join('.', 'data', 'mini_speech_commands_silence')
else:
    zip_path = tf.keras.utils.get_file(
        origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
        fname='mini_speech_commands.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')

    data_dir = os.path.join('.', 'data', 'mini_speech_commands')

filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)

#if args.silence is True:
if silence is True:
    total = 9000
else:
    total = 8000

182083584/182082353 [==============================] - 10s 0us/step


#Training, Validation and Test dataset definition

In [6]:
train_files = filenames[:int(total*0.8)]
val_files = filenames[int(total*0.8): int(total*0.9)]
test_files = filenames[int(total*0.9):]

In [7]:
LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']

#Data preparation and preprocessing steps

In [8]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds

In [15]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 8, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}

In [10]:
#if args.mfcc is True:
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]


In [11]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)

#if args.silence is True:
if silence is True:
    units = 9
else:
    units = 8

#Model application

##MPL

In [12]:
#MLP Model definition
mlp_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=8) 
])

In [14]:
#Training, evaluation and testing with mlp model


mlp_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
mlp_model.fit(train_ds, epochs=20)

Epoch 1/20
200/200 [==============================] - 102s 511ms/step - loss: 1.7112 - accuracy: 0.4489
Epoch 2/20
200/200 [==============================] - 1s 3ms/step - loss: 0.9281 - accuracy: 0.6667
Epoch 3/20
200/200 [==============================] - 1s 3ms/step - loss: 0.7310 - accuracy: 0.7378
Epoch 4/20
200/200 [==============================] - 1s 3ms/step - loss: 0.5718 - accuracy: 0.7969
Epoch 5/20
200/200 [==============================] - 1s 3ms/step - loss: 0.4671 - accuracy: 0.8367
Epoch 6/20
200/200 [==============================] - 1s 3ms/step - loss: 0.4056 - accuracy: 0.8550
Epoch 7/20
200/200 [==============================] - 1s 3ms/step - loss: 0.3443 - accuracy: 0.8772
Epoch 8/20
200/200 [==============================] - 1s 3ms/step - loss: 0.3003 - accuracy: 0.8934
Epoch 9/20
200/200 [==============================] - 1s 4ms/step - loss: 0.2867 - accuracy: 0.9031
Epoch 10/20
200/200 [==============================] - 1s 3ms/step - loss: 0.2815 - accuracy: 0.

In [18]:
#model evaluation on validation set
mlp_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= mlp_model.evaluate(val_ds)

25/25 [==============================] - 0s 1ms/step - loss: 1.7222


In [25]:
test_loss= mlp_model.evaluate(test_ds)

##CNN

In [21]:
#CNN_2D Model definition

cnn_2d_model = keras.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [22]:
#Training, evaluation and testing with mlp model
cnn_2d_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
cnn_2d_model.fit(train_ds, epochs=20)

Epoch 1/20
200/200 [==============================] - 32s 159ms/step - loss: 1.2155 - accuracy: 0.6336
Epoch 2/20
200/200 [==============================] - 29s 146ms/step - loss: 0.6260 - accuracy: 0.8400
Epoch 3/20
200/200 [==============================] - 29s 146ms/step - loss: 0.4398 - accuracy: 0.8852
Epoch 4/20
200/200 [==============================] - 29s 146ms/step - loss: 0.3530 - accuracy: 0.9045
Epoch 5/20
200/200 [==============================] - 29s 146ms/step - loss: 0.2886 - accuracy: 0.9228
Epoch 6/20
200/200 [==============================] - 29s 146ms/step - loss: 0.2477 - accuracy: 0.9330
Epoch 7/20
200/200 [==============================] - 30s 148ms/step - loss: 0.2093 - accuracy: 0.9431
Epoch 8/20
200/200 [==============================] - 29s 146ms/step - loss: 0.1855 - accuracy: 0.9492
Epoch 9/20
200/200 [==============================] - 29s 146ms/step - loss: 0.1591 - accuracy: 0.9569
Epoch 10/20
200/200 [==============================] - 30s 152ms/step - l

In [26]:
#model evaluation on validation set
cnn_2d_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= cnn_2d_model.evaluate(val_ds)

25/25 [==============================] - 1s 36ms/step - loss: 0.7952


In [27]:
#model test on the test set
test_loss= cnn_2d_model.evaluate(test_ds)

25/25 [==============================] - 1s 35ms/step - loss: 0.8806


##DS-CNN

In [28]:
ds_cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=256,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [29]:
#Training, evaluation and testing with mlp model
ds_cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
ds_cnn_model.fit(train_ds, epochs=20)

Epoch 1/20
200/200 [==============================] - 22s 108ms/step - loss: 1.2386 - accuracy: 0.6083
Epoch 2/20
200/200 [==============================] - 22s 109ms/step - loss: 0.6136 - accuracy: 0.8327
Epoch 3/20
200/200 [==============================] - 22s 109ms/step - loss: 0.4223 - accuracy: 0.8858
Epoch 4/20
200/200 [==============================] - 22s 109ms/step - loss: 0.3319 - accuracy: 0.9081
Epoch 5/20
200/200 [==============================] - 22s 109ms/step - loss: 0.2846 - accuracy: 0.9148
Epoch 6/20
200/200 [==============================] - 22s 109ms/step - loss: 0.2454 - accuracy: 0.9292
Epoch 7/20
200/200 [==============================] - 22s 109ms/step - loss: 0.2074 - accuracy: 0.9419
Epoch 8/20
200/200 [==============================] - 22s 110ms/step - loss: 0.1818 - accuracy: 0.9477
Epoch 9/20
200/200 [==============================] - 22s 111ms/step - loss: 0.1600 - accuracy: 0.9555
Epoch 10/20
200/200 [==============================] - 22s 110ms/step - l

In [30]:
#model evaluation on validation set
ds_cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= ds_cnn_model.evaluate(val_ds)

25/25 [==============================] - 1s 28ms/step - loss: 0.7021


In [31]:
#model test on the test set
test_loss= ds_cnn_model.evaluate(test_ds)

25/25 [==============================] - 1s 28ms/step - loss: 0.6723
